# Reading, Writing and Validating Data in PySpark HW

Welcome to your first coding homework assignment in PySpark! I hope you enjoyed the lecture on Reading, Writing and Validating dataframes. Now it's time to put what you've learned into action! 

I've included several instructions below to help guide you through this homework assignment which I hope will get you feeling even comfortable reading, writing and validating dataframes. If you get stuck at any point, feel free to jump to the next lecture where I will guide you through my solutions to the HW assignment. 

Have fun!

Let's dig right in!


## But first things first.....
We need to always begin every Spark session by creating a Spark instance. Let's go ahead and use the method we learned in the lecture in the cell below. Also see if you can remember how to open the Spark UI (using a link that automatically guides you there). 

In [2]:
import pandas as pd
import numpy as np
import datetime as dt

import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *#avg, count, expr
from pyspark.sql.types import *

In [3]:
# initialize
sc = pyspark.SparkContext()
spark = SparkSession(sc)
spark.sparkContext.appName = 'exploreReadWrite'
# show the number of cores
print('%d cores'%spark._jsc.sc().getExecutorMemoryStatus().keySet().size())
spark

1 cores


## Next let's start by reading a basic csv dataset

Download the pga_tour_historical dataset that is attached to this lecture and save it whatever folder you want, then read it in. 

**Data Source:** https://www.kaggle.com/bradklassen/pga-tour-20102018-data

Rememer to try letting Spark infer the header and infer the Schema types!

In [5]:
# load the file
fil = '../../data/pga_tour_historical.csv'

pgaData = spark.read.format('csv').options(header=True, inferSchema=True).load(fil)

## 1. View first 5 lines of dataframe
First generate a view of the first 5 lines of the dataframe to get an idea of what is inside. We went over two ways of doing this... see if you can remember BOTH ways. 

In [8]:
display(pgaData.limit(5).toPandas())

,Player Name,Season,Statistic,Variable,Value
0,Robert Garrigus,2010,Driving Distance,Driving Distance - (ROUNDS),71
1,Bubba Watson,2010,Driving Distance,Driving Distance - (ROUNDS),77
2,Dustin Johnson,2010,Driving Distance,Driving Distance - (ROUNDS),83
3,Brett Wetterich,2010,Driving Distance,Driving Distance - (ROUNDS),54
4,J.B. Holmes,2010,Driving Distance,Driving Distance - (ROUNDS),100


## 2. Print the schema details

Now print the details of the dataframes schema that Spark infered to ensure that it was infered correctly. Sometimes it is not infered correctly, so we need to watch out!

In [7]:
pgaData.printSchema()

root
 |-- Player Name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: string (nullable = true)



## 3. Edit the schema during the read in

We can see from the output above that Spark did not correctly infer that the "value" column was an integer value. Let's try specifying the schema this time to let spark know what the schema should be.

Here is a link to see a list of PySpark data types in case you need it (also attached to the lecture): 
https://spark.apache.org/docs/latest/sql-ref-datatypes.html

In [9]:
# define the schema
schema = StructType([StructField('Player', StringType()),
                     StructField('Season', IntegerType()),
                     StructField('Statistic', StringType()),
                     StructField('Variable', StringType()),
                     StructField('Value', IntegerType())])
# reload the data
pgaData = spark.read.format('csv').options(header=True, inferSchema=False).schema(schema).load(fil)

## 4. Generate summary statistics for only one variable

See if you can generate summary statistics for only the "Value" column using the .describe function

(count, mean, stddev, min, max) 

In [11]:
pgaData.select('Value').summary('count', 'min', 'mean', 'max', 'stddev').show()

+-------+------------------+
|summary|             Value|
+-------+------------------+
|  count|           1657247|
|    min|              -178|
|   mean|12494.388998743096|
|    max|           3564954|
| stddev|157274.75673570775|
+-------+------------------+



## 5. Generate summary statistics for TWO variables
Now try to generate ONLY the count min and max for BOTH the "Value" and "Season" variable using the select. You can't use the .describe function for this one but see if you can remember which function you CAN use. 

In [13]:
pgaData.select('Season', 'Value').agg(min('Season').alias('First_season'), min('Value').alias('Lowest'),
                                      max('Season').alias('Last_season'), max('Value').alias('Highest')).show()

+------------+------+-----------+-------+
|First_season|Lowest|Last_season|Highest|
+------------+------+-----------+-------+
|        2010|  -178|       2018|3564954|
+------------+------+-----------+-------+



## 6. Write a parquet file

Now try writing a parquet file (not partitioned) from the pga dataset. But first create a new dataframe containing ONLY the the "Season" and "Value" fields (using the "select command you used in the question above) and write a parquet file partitioned by "Season". This is a bit of a challenge aimed at getting you ready for material that will be covered later on in the course. Don't feel bad if you can't figure it out.

*Note that if any of your variable names contain spaces, spark will produce an error message with this call. That is why we are selecting ONLY the "Season" and "Value" fields. Ideally we should renamed those columns but we haven't gotten to that yet in this course but we will soon!*

In [14]:
fil = '../../data/pgaDataSeason'
pgaData.select('Season', 'Value').write.format('parquet').partitionBy('Season').save(fil)

## 7. Write a partioned parquet file

You will need to use the same limited dataframe that you created in the previous question to accomplish this task as well. 

In [15]:
# don't get this, as the previous section clearly said to partition the output...

## 8. Read in a partitioned parquet file

Now try reading in the partitioned parquet file you just created above. 

In [19]:
# get data
fil = '../../data/pgaDataSeason'
pgaData = spark.read.format('parquet').load(fil)
# count records by season
pgaData.groupBy('Season').agg(count('Season').alias('Records'), mean('Value').alias('MeanVal')).orderBy('Season').show()

+------+-------+------------------+
|Season|Records|           MeanVal|
+------+-------+------------------+
|  2010| 323083|11358.987663492031|
|  2011| 305112|12309.877901710195|
|  2012| 311658|11892.438257226466|
|  2013| 291958|12252.976537822697|
|  2014| 292478| 13067.02860306644|
|  2015| 302357|13167.718784867455|
|  2016| 300549|12753.346759992713|
|  2017| 306023|12216.024854215328|
|  2018| 307185| 13525.16256894057|
+------+-------+------------------+



## 9. Reading in a set of paritioned parquet files

Now try only reading Seasons 2010, 2011 and 2012.

In [31]:
# get a subset of the data
fil = '../../data/pgaDataSeason/'
pgaData = spark.read.options(basepath=fil).parquet([fil+'Season=2010', fil+'Season=2011', fil+'Season=2012'])
# count records by season
pgaData.groupBy('Season').agg(count('Season').alias('Records'), mean('Value').alias('MeanVal')).orderBy('Season').show()

Py4JJavaError: An error occurred while calling o268.parquet.
: java.lang.ClassCastException: java.util.ArrayList cannot be cast to java.lang.String
	at org.apache.spark.sql.internal.SessionState.$anonfun$newHadoopConfWithOptions$1(SessionState.scala:102)
	at org.apache.spark.sql.internal.SessionState.$anonfun$newHadoopConfWithOptions$1$adapted(SessionState.scala:102)
	at scala.collection.immutable.Map$Map2.foreach(Map.scala:159)
	at org.apache.spark.sql.internal.SessionState.newHadoopConfWithOptions(SessionState.scala:102)
	at org.apache.spark.sql.execution.datasources.DataSource.newHadoopConfiguration(DataSource.scala:114)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:375)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:326)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$3(DataFrameReader.scala:308)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.

## 10. Create your own dataframe

Try creating your own dataframe below using PySparks *.createDataFrame* function. See if you can make one that contains 4 variables and at least 3 rows. 

Let's see how creative you can get on the content of the dataframe :)

In [21]:
n, p = (10, 4)
data = np.random.rand(n, p)

# define the schema
schem = StructType([StructField('col_%d'%i, FloatType(), False) for i in range(p)])

# create the dataframe
randData = spark.createDataFrame(data.tolist(), schema=schem)
randData.show()

Py4JJavaError: An error occurred while calling o221.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 34.0 failed 1 times, most recent failure: Lost task 0.0 in stage 34.0 (TID 619) (192.168.150.128 executor driver): org.apache.spark.SparkException: 
Bad data in pyspark.daemon's standard output. Invalid port number:
  458961458 (0x1b5b3232)
Python command to execute the daemon was:
  ipython3 -m pyspark.daemon
Check that you don't have any unexpected modules or libraries in
your PYTHONPATH:
  /home/ahowe42/spark-3.1.1-bin-hadoop2.7/python/lib/pyspark.zip:/home/ahowe42/spark-3.1.1-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip:/home/ahowe42/spark-3.1.1-bin-hadoop2.7/jars/spark-core_2.12-3.1.1.jar:/home/ahowe42/spark-3.1.1-bin-hadoop2.7/python:
Also, check if you have a sitecustomize.py module in your python path,
or in your python installation, that is printing to standard output
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon

## We're done! Great job!

In [ ]:
sc.stop()